In [1]:
import numpy as np
import matplotlib.pyplot as plt

# material flow

source -> **sourceout** -> enrichment -> **enrichmentout** -> lwr -> **lwrout** -> lwrstorage -> **lwrstorageout** -> lwrreprocessing -> **lwrreprocessingoutpu, lwrreprocessingoutu** -> pumixer -> **pumixerout** -> fr -> **frout** -> frstorage -> **frstorageout** -> frreprocessing -> **frreprocessingoutpu, frreprocessingoutu** -> pumixer

In [2]:
# simulation parameters
total_duration = 120*12
# lazy, manual, never
decay = 'lazy'
calc_method = 'ma'


# transition parameters
transition_begin_timestep = 80*12 # timestep to start transition
transition_duration = 100
stages_of_decom = 10 # number of decommissioning `steps'
power_demand = 60*10**3 # [MWe]
lwr_power = 1*10**3 #MWe
fr_power = 0.333*10**3
fr_lifetime = 60 * 12


# mixer parameters
mixer_throughput = '1e6'
mixer_out_buf_size = '1e6'

# reprocessing parameters
reprocessing_feed_buf_size = '1e8'
lwr_reprocessing_throughput = '1e8'
fr_reprocessing_throughput = '1e7'
efficiency = 1

# enrichment parameters
enrichment_tails_assay = 0.0025
enrichment_swu_capacity = '1e100'

# source parameters
source_throughput = '1e10'

# storage parameters
lwr_storage_capacity = '1e8'
lwr_residence_time = 60
fr_storage_capacity = '1e8'
fr_residence_time = 36

# sink parameters
lwrsinkcapacity = '1e6' 
frsinkcapacity = '1e6'

# prototype definitions

In [3]:
# reactors
lwr = """
<facility>
        <name>lwr</name>
        <config>
            <Reactor>
                <fuel_inrecipes>
                    <val>lwrinrecipe</val>
                </fuel_inrecipes>
                <fuel_outrecipes>
                    <val>lwroutrecipe</val>
                </fuel_outrecipes>
                <fuel_incommods>
                    <val>enrichmentout</val>
                </fuel_incommods>
                <fuel_outcommods>
                    <val>lwrout</val>
                </fuel_outcommods>
                <cycle_time>18</cycle_time>
                <refuel_time>0</refuel_time>
                <assem_size>29863.3</assem_size>
                <n_assem_core>3</n_assem_core>
                <n_assem_batch>1</n_assem_batch>
                <power_cap>1000</power_cap>
            </Reactor>
        </config>
    </facility>
"""

###########SHOULD MAKE MOX LWR########################

fr = """
<facility>
        <name>fr</name>
        <lifetime>%i</lifetime>
        <config>
            <Reactor>
                <fuel_inrecipes>
                    <val>frinrecipe</val>
                </fuel_inrecipes>
                <fuel_outrecipes>
                    <val>froutrecipe</val>
                </fuel_outrecipes>
                <fuel_incommods>
                    <val>pumixerout</val>
                </fuel_incommods>
                <fuel_outcommods>
                    <val>frout</val>
                </fuel_outcommods>
                <cycle_time>12</cycle_time>
                <refuel_time>0</refuel_time>
                <assem_size>3950</assem_size>
                <n_assem_core>4</n_assem_core>
                <n_assem_batch>1</n_assem_batch>
                <power_cap>333.3</power_cap>
            </Reactor>
        </config>
    </facility>
""" %fr_lifetime

In [4]:
# mixers
pumixerlwr = """
<facility>
        <name>pumixerlwr</name>
        <config>
            <Mixer>
                <in_streams>
                    <stream>
                        <info>
                            <mixing_ratio>0.1387</mixing_ratio>
                            <buf_size>%s</buf_size>
                        </info>
                        <commodities>
                            
                            <item>
                                <commodity>lwrreprocessingoutpu</commodity>
                                <pref>1.0</pref>
                            </item>
                        </commodities>
                    </stream>
                    <stream>
                        <info>
                            <mixing_ratio>0.8613</mixing_ratio>
                            <buf_size>%s</buf_size>
                        </info>
                        <commodities>
                            
                            <item>
                                <commodity>lwrreprocessingoutu</commodity>
                                <pref>1.0</pref>
                            </item>
                        </commodities>
                    </stream>
                </in_streams>
                <out_commod>pumixerout</out_commod>
                <out_buf_size>%s</out_buf_size>
                <throughput>%s</throughput>
            </Mixer>
        </config>
    </facility>
""" %(mixer_out_buf_size, mixer_out_buf_size, mixer_out_buf_size, mixer_throughput)

pumixerfr = """
<facility>
        <name>pumixerfr</name>
        <config>
            <Mixer>
                <in_streams>
                    <stream>
                        <info>
                            <mixing_ratio>0.1387</mixing_ratio>
                            <buf_size>%s</buf_size>
                        </info>
                        <commodities>
                            
                            <item>
                                <commodity>frreprocessingoutpu</commodity>
                                <pref>1.0</pref>
                            </item>
                        </commodities>
                    </stream>
                    <stream>
                        <info>
                            <mixing_ratio>0.8613</mixing_ratio>
                            <buf_size>%s</buf_size>
                        </info>
                        <commodities>
                            
                            <item>
                                <commodity>frreprocessingoutu</commodity>
                                <pref>1.0</pref>
                            </item>
                        </commodities>
                    </stream>
                </in_streams>
                <out_commod>pumixerout</out_commod>
                <out_buf_size>%s</out_buf_size>
                <throughput>%s</throughput>
            </Mixer>
        </config>
    </facility>
""" %(mixer_out_buf_size, mixer_out_buf_size, mixer_out_buf_size, mixer_throughput)

############ add tru mixer###########


In [5]:
# reprocessing
lwrreprocessing_pu = """
    <facility>
        <name>lwrreprocessing</name>
        <config>
            <Separations>
                <feed_commods>
                    <val>lwrstorageout</val>
                </feed_commods>
                <feedbuf_size>%s</feedbuf_size>
                <throughput>%s</throughput>
                <leftover_commod>lwrreprocessingwaste</leftover_commod>
                <leftoverbuf_size>%s</leftoverbuf_size>
                <streams>
                    <item>
                        <commod>lwrreprocessingoutpu</commod>
                        <info>
                            <buf_size>%s</buf_size>
                            <efficiencies>
                                <item>
                                    <comp>Pu</comp>
                                    <eff>%s</eff>
                                </item>
                            </efficiencies>
                        </info>
                    </item>
                    <item>
                        <commod>lwrreprocessingoutu</commod>
                        <info>
                            <buf_size>%s</buf_size>
                            <efficiencies>
                                <item>
                                    <comp>U</comp>
                                    <eff>%s</eff>
                                </item>
                            </efficiencies>
                        </info>
                    </item>
                </streams>
            </Separations>
        </config>
    </facility>
""" %(reprocessing_feed_buf_size, lwr_reprocessing_throughput, reprocessing_feed_buf_size,
      reprocessing_feed_buf_size, efficiency, reprocessing_feed_buf_size, efficiency)


frreprocessing_pu = """
    <facility>
        <name>frreprocessing</name>
        <config>
            <Separations>
                <feed_commods>
                    <val>frstorageout</val>
                </feed_commods>
                <feedbuf_size>%s</feedbuf_size>
                <throughput>%s</throughput>
                <leftover_commod>frreprocessingwaste</leftover_commod>
                <leftoverbuf_size>%s</leftoverbuf_size>
                <streams>
                    <item>
                        <commod>frreprocessingoutpu</commod>
                        <info>
                            <buf_size>%s</buf_size>
                            <efficiencies>
                                <item>
                                    <comp>Pu</comp>
                                    <eff>%s</eff>
                                </item>
                            </efficiencies>
                        </info>
                    </item>
                    <item>
                        <commod>frreprocessingoutu</commod>
                        <info>
                            <buf_size>%s</buf_size>
                            <efficiencies>
                                <item>
                                    <comp>U</comp>
                                    <eff>%s</eff>
                                </item>
                            </efficiencies>
                        </info>
                    </item>
                </streams>
            </Separations>
        </config>
    </facility>
""" %(reprocessing_feed_buf_size, fr_reprocessing_throughput, reprocessing_feed_buf_size,
      reprocessing_feed_buf_size, efficiency, reprocessing_feed_buf_size, efficiency)


###########SHOULD MAKE TRU reprocessing########################

In [6]:
enrichment = """
    <facility>
        <name>enrichment</name>
        <config>
            <Enrichment>
                <feed_commod>sourceout</feed_commod>
                <feed_recipe>sourceoutrecipe</feed_recipe>
                <product_commod>enrichmentout</product_commod>
                <tails_assay>%s</tails_assay>
                <tails_commod>enrichmentwaste</tails_commod>
                <swu_capacity>%s</swu_capacity>
                <initial_feed>0</initial_feed>
            </Enrichment>
        </config>
    </facility>
"""%(enrichment_tails_assay, enrichment_swu_capacity)

In [7]:
source = """
<facility>
    <name>source</name>
    <config>
        <Source>
            <outcommod>sourceout</outcommod>
            <outrecipe>sourceoutrecipe</outrecipe>
            <throughput>%s</throughput>
        </Source>
    </config>
</facility>
""" %source_throughput

In [8]:
lwrstorage = """

    <facility>
        <name>lwrstorage</name>
        <config>
            <Storage>
                <in_commods>
                    <val>lwrout</val>
                </in_commods>
                <residence_time>%s</residence_time>
                <out_commods>
                    <val>lwrstorageout</val>
                </out_commods>
                <max_inv_size>%s</max_inv_size>
            </Storage>
        </config>
    </facility>
""" %(lwr_residence_time, lwr_storage_capacity)

frstorage = """
    <facility>
        <!-- Stores uox unf for 48 months (4years) -->
        <name>frstorage</name>
        <config>
            <Storage>
                <in_commods>
                    <val>frout</val>
                </in_commods>
                <residence_time>%s</residence_time>
                <out_commods>
                    <val>frstorageout</val>
                </out_commods>
                <max_inv_size>%s</max_inv_size>
            </Storage>
        </config>
    </facility>
""" %(fr_residence_time, fr_storage_capacity)

###########SHOULD MAKE MOX Storage########################

In [9]:
# sinks
lwrsink = """
<facility>
        <name>lwrsink</name>
        <config>
            <Sink>
                <in_commods>
                    <val>lwrreprocessingwaste</val>
                </in_commods>
                <capacity>%s</capacity>
            </Sink>
        </config>
    </facility>
""" %lwrsinkcapacity
frsink = """
<facility>
        <name>frsink</name>
        <config>
            <Sink>
                <in_commods>
                    <val>frreprocessingwaste</val>
                </in_commods>
                <capacity>%s</capacity>
            </Sink>
        </config>
    </facility>
""" %frsinkcapacity

# deploy scheme (region definition)

In [10]:
lwr_lifetimes = np.arange(transition_begin_timestep, transition_begin_timestep+transition_duration, stages_of_decom)
lwr_initial_facility_list = """"""
for num in range(1,len(lwr_lifetimes)+1):
    total_lwr = power_demand / lwr_power
    lwr_per_stage = total_lwr / len(lwr_lifetimes)
    lwr_initial_facility_list_temp = """
    <institution>
        <name>initialfacilitylist_%i</name>
        <lifetime>%i</lifetime>
        <initialfacilitylist> 
            <entry>
                <prototype>lwr</prototype> 
                <number>%i</number>
            </entry>
        </initialfacilitylist>
        <config><NullInst/></config>
    </institution>
    """ %(num, lwr_lifetimes[num-1],lwr_per_stage)
    lwr_initial_facility_list = lwr_initial_facility_list + lwr_initial_facility_list_temp

In [11]:
lwr_lifetimes = np.arange(transition_begin_timestep, transition_begin_timestep+transition_duration, stages_of_decom)
total_lwr = power_demand / lwr_power
lwr_per_stage = total_lwr / len(lwr_lifetimes)

lwr_inst_def = """
<institution> 
    <name>lwr_inst</name>
    <config>
        <DeployInst>
            <prototypes>\n"""
for j in range(stages_of_decom):
    lwr_inst_def += "\t\t\t<val>lwr</val> \n"
lwr_inst_def += "\t\t</prototypes> \n \t\t<build_times>\n"
for j in range(stages_of_decom):
    lwr_inst_def += "\t\t\t<val>1</val> \n"
lwr_inst_def += "\t\t</build_times> \n \t\t<n_build>\n"
for j in range(stages_of_decom):
    lwr_inst_def += "\t\t\t<val>%i</val> \n" %lwr_per_stage
lwr_inst_def += "\t\t</n_build> \n \t\t<lifetimes>\n"
for lifetime in lwr_lifetimes:
    lwr_inst_def += "\t\t\t<val>%i</val> \n" %lifetime
lwr_inst_def += "\t\t</lifetimes>\n\t\t</DeployInst>\n\t</config>\n </institution>"



#########################################
# no need to use if using timeseries inst
fr_inst_def = """
<institution> 
    <name>fr_inst</name>
    <config>
        <DeployInst>
            <prototypes>\n"""
for j in range(stages_of_decom):
    fr_inst_def += "\t\t\t<val>fr</val> \n"
fr_inst_def += "\t\t</prototypes> \n \t\t<build_times>\n"
for lifetime in lwr_lifetimes:
    fr_inst_def += "\t\t\t<val>%i</val> \n" %lifetime
fr_inst_def += "\t\t</build_times> \n \t\t<n_build>\n"
for j in range(stages_of_decom):
    fr_inst_def += "\t\t\t<val>%i</val> \n" %(lwr_per_stage * int(lwr_power / fr_power))
fr_inst_def += "\t\t</n_build> \n \t\t<lifetimes>\n"
for j in range(stages_of_decom):
    fr_inst_def += "\t\t\t<val>%i</val> \n" %fr_lifetime
fr_inst_def += "\t\t</lifetimes>\n\t\t</DeployInst>\n\t</config>\n </institution>"
print(lwr_inst_def)
print(fr_inst_def)


<institution> 
    <name>lwr_inst</name>
    <config>
        <DeployInst>
            <prototypes>
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
			<val>lwr</val> 
		</prototypes> 
 		<build_times>
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
			<val>1</val> 
		</build_times> 
 		<n_build>
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
			<val>6</val> 
		</n_build> 
 		<lifetimes>
			<val>960</val> 
			<val>970</val> 
			<val>980</val> 
			<val>990</val> 
			<val>1000</val> 
			<val>1010</val> 
			<val>1020</val> 
			<val>1030</val> 
			<val>1040</val> 
			<val>1050</val> 
		</lifetimes>
		</DeployInst>
	</config>
 </institution>

<institution> 
    <

In [12]:
# supply chain
# timeseries_inst definition
timeseries_inst = """
<institution>
<config>
    <TimeSeriesInst>
        <calc_method>%s</calc_method>
        <demand_eq>60000</demand_eq>
        <commodities>
            <val>enrichmentout_enrichment_%s</val>
            <val>sourceout_source_%s</val>
            <val>POWER_fr_%s_t-840</val>
        </commodities>
        <demand_std_dev>0</demand_std_dev>
        <steps>1</steps>
    </TimeSeriesInst>
</config>
<name>timeseriesinst</name>
</institution>
""" %(calc_method, float(source_throughput) * 1e100, source_throughput, fr_power)

supplydrivendepolymentinst = """
<institution>
<config>
    <SupplyDrivenDeploymentInst>
        <calc_method>%s</calc_method>
        <commodities>
            <val>lwrstorageout_lwrreprocessing_%s</val>
            <val>frstorageout_frreprocessing_%s</val>
            <val>lwrout_lwrstorage_%s</val>
            <val>frout_frstorage_%s</val>
            <val>lwrreprocessingoutpu_pumixerlwr_%s</val>
            <val>frreprocessingoutpu_pumixerfr_%s</val>
            <val>lwrreprocessingwaste_lwrsink_%s</val>
            <val>frreprocessingwaste_frsink_%s</val>
        </commodities>
        <capacity_std_dev>0</capacity_std_dev>
        <steps>1</steps>
    </SupplyDrivenDeploymentInst>
</config>
<name>supplydrivendeploymentinst</name>
</institution>
""" %(calc_method, lwr_reprocessing_throughput, fr_reprocessing_throughput,
      lwr_storage_capacity, fr_storage_capacity, mixer_throughput, mixer_throughput,
      lwrsinkcapacity, frsinkcapacity)

In [13]:
control = """
<control>
    <duration>%i</duration>
    <startmonth>1</startmonth>
    <startyear>2000</startyear>
    <decay>%s</decay>
</control>""" %(total_duration, decay)


In [14]:
archetypes = """
<archetypes>
        <spec>
            <lib>cycamore</lib>
            <name>Source</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>Sink</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>Reactor</name>
        </spec>
        <spec>
            <lib>agents</lib>
            <name>NullRegion</name>
        </spec>
        <spec>
            <lib>agents</lib>
            <name>NullInst</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>DeployInst</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>Separations</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>Enrichment</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>Mixer</name>
        </spec>
        <spec>
            <lib>cycamore</lib>
            <name>Storage</name>
        </spec>
        <spec>
            <lib>d3ploy.timeseries_inst</lib>
            <name>TimeSeriesInst</name>
        </spec>
        <spec>
            <lib>d3ploy.supply_driven_deployment_inst</lib>
            <name>SupplyDrivenDeploymentInst</name>
        </spec>
    </archetypes>
"""

In [15]:
proto_definitions = lwr + fr + pumixerlwr + pumixerfr + lwrreprocessing_pu + frreprocessing_pu + enrichment + source + lwrstorage + frstorage + lwrsink + frsink

In [16]:
region_begin = """
<region>
    <config>
        <NullRegion>
        </NullRegion>
    </config>
"""
region_end = """
<name>region</name>
</region>
"""
region = region_begin + lwr_inst_def + timeseries_inst + supplydrivendepolymentinst + region_end

In [17]:
recipe = """
    <recipe>
        <name>sourceoutrecipe</name>
        <basis>mass</basis>
        <nuclide>
            <id>U235</id>
            <comp>0.711</comp>
        </nuclide>
        <nuclide>
            <id>U238</id>
            <comp>99.289</comp>
        </nuclide>
    </recipe>

    <recipe>
        <name>lwrinrecipe</name>
        <basis>mass</basis>
        <nuclide>
            <id>U235</id>
            <comp>4.3</comp>
        </nuclide>
        <nuclide>
            <id>U238</id>
            <comp>95.7</comp>
        </nuclide>
    </recipe>

    <recipe>
        <name>lwroutrecipe</name>
        <basis>mass</basis>
        <!-- U is generalized to u238-->
        <nuclide>
            <id>U238</id>
            <comp>93.44</comp>
        </nuclide>
        <!-- TRU is generalized to pu239-->
        <nuclide>
            <id>Pu239</id>
            <comp>1.30</comp>
        </nuclide>
        <!-- FP is generalized to cs137-->
        <nuclide>
            <id>Cs137</id>
            <comp>5.26</comp>
        </nuclide>
    </recipe>

    <recipe>
        <name>froutrecipe</name>
        <basis>mass</basis>
        <!-- U is generalized to u238-->
        <nuclide>
            <id>U238</id>
            <comp>78.30</comp>
        </nuclide>
        <!-- TRU is generalized to pu239-->
        <nuclide>
            <id>Pu239</id>
            <comp>14.04</comp>
        </nuclide>
        <!-- FP is generalized to cs137-->
        <nuclide>
            <id>Cs137</id>
            <comp>7.66</comp>
        </nuclide>
    </recipe>

    <recipe>
        <name>frinrecipe</name>
        <basis>mass</basis>
        <!-- U is generalized to u238-->
        <nuclide>
            <id>U238</id>
            <comp>86.13</comp>
        </nuclide>
        <!-- TRU is generalized to pu239-->
        <nuclide>
            <id>Pu239</id>
            <comp>13.87</comp>
        </nuclide>
    </recipe>
"""

In [18]:
with open('input.xml', 'w') as f:
    f.write('<simulation>\n')
    f.write(control + archetypes)
    f.write(proto_definitions)
    f.write(region)
    f.write(recipe)
    f.write('</simulation>')